In [ ]:
from google.colab import files

# Upload the dataset ZIP file manually
uploaded = files.upload()


In [ ]:
import zipfile
import os

zip_path = "/content/archive.zip"  # Change name if yours is different
extract_path = "/content/garbage_dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

os.listdir(extract_path)  # Check folders


In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

original_data_dir = "/content/garbage_dataset/TrashType_Image_Dataset"
base_dir = "/content/garbage_dataset_split"

train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

# Create folders again if not exist
for folder in [train_dir, val_dir, test_dir]:
    os.makedirs(folder, exist_ok=True)

# Split images per class
categories = os.listdir(original_data_dir)

for category in categories:
    category_path = os.path.join(original_data_dir, category)
    images = os.listdir(category_path)

    train_imgs, temp_imgs = train_test_split(images, test_size=0.3, random_state=42)
    val_imgs, test_imgs = train_test_split(temp_imgs, test_size=0.5, random_state=42)

    for split_type, split_imgs in zip(['train', 'val', 'test'], [train_imgs, val_imgs, test_imgs]):
        split_folder = os.path.join(base_dir, split_type, category)
        os.makedirs(split_folder, exist_ok=True)

        for img in split_imgs:
            src = os.path.join(category_path, img)
            dst = os.path.join(split_folder, img)
            shutil.copy(src, dst)

print("✅ Dataset re-split into train, val, test.")



In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# =========================
# ⚙️ Prepare the Data
# =========================
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/garbage_dataset_split/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    '/content/garbage_dataset_split/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# =========================
# 🧠 Load MobileNetV2 Base
# =========================
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base

# =========================
# 🧱 Build the Model Head
# =========================
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# =========================
# ⚙️ Compile the Model
# =========================
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# =========================
# ⏱️ Setup Callbacks
# =========================
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_reduce = ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.5)

# =========================
# 🚀 Train the Model
# =========================
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[early_stop, lr_reduce]
)


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Parameters
IMAGE_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 20

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.4)(x)
predictions = Dense(train_gen.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

history = model.fit(
    train_gen,
    epochs=EPOCHS,
    validation_data=val_gen
)


In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Over Epochs')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Over Epochs')
plt.legend()

plt.show()


In [ ]:
model.save("/content/garbage_classifier_mobilenetv2.h5")
print("✅ Model saved.")


In [ ]:
from tensorflow.keras.models import load_model

# Load the model you saved earlier
model = load_model("/content/garbage_classifier_mobilenetv2.h5")
print("✅ Model loaded.")


In [ ]:
from google.colab import files
from IPython.display import display
import PIL.Image

uploaded = files.upload()

for fn in uploaded.keys():
    print(f"📸 Uploaded file: {fn}")
    img = PIL.Image.open(fn)
    display(img)

    # Predict
    preprocess_and_predict(fn, model, class_indices)


In [ ]:
train_generator.class_indices


In [ ]:
val_loss, val_acc = model.evaluate(val_generator)
print(f"📊 Final Val Accuracy: {val_acc:.2f}")


In [ ]:
model.save("garbage_classifier_mobilenetv2.h5")


In [ ]:
from keras.models import load_model

# Load your trained model
model = load_model("garbage_classifier_mobilenetv2.h5")
print("✅ Model loaded.")


In [ ]:
from google.colab import files
from keras.preprocessing import image
import numpy as np
import PIL
import matplotlib.pyplot as plt

# Step 1: Upload an image
uploaded = files.upload()

# Step 2: Get the uploaded filename
filename = list(uploaded.keys())[0]
print(f"✅ Uploaded: {filename}")

# Step 3: Preprocess the image
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))  # resize to model's input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # add batch dimension
    img_array = img_array / 255.0  # normalize
    return img_array

# Step 4: Predict and Show
def predict_image(img_path, model):
    img_array = preprocess_image(img_path)
    prediction = model.predict(img_array)
    predicted_class_index = np.argmax(prediction, axis=1)[0]

    # Get class labels
    class_labels = list(train_generator.class_indices.keys())  # assumes train_generator was used earlier
    predicted_label = class_labels[predicted_class_index]

    # Show image and prediction
    img = PIL.Image.open(img_path)
    plt.imshow(img)
    plt.axis('off')
    plt.title(f"Prediction: {predicted_label}")
    plt.show()

# Step 5: Call prediction
predict_image(filename, model)


In [ ]:
from keras.models import load_model
model = load_model('garbage_classifier_mobilenetv2.h5')  # e.g. model.h5
